In [52]:
import pandas as pd
data = pd.read_csv('../CSV/FullMain.csv')
data[data['<TICKER>'] == 'ALRS'].head()

,<TICKER>,<DATETIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
7790,ALRS,2021-01-11 11:00:00,104.59,105.48,102.67,105.26,4359830
7791,ALRS,2021-01-11 12:00:00,105.26,105.59,104.56,104.85,2504390
7792,ALRS,2021-01-11 13:00:00,104.81,104.92,103.70,104.46,1887640
7793,ALRS,2021-01-11 14:00:00,104.45,105.19,104.39,104.83,3060770
7794,ALRS,2021-01-11 15:00:00,104.83,107.27,104.32,106.35,7179830


In [53]:
import numpy as np
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score, precision_score, mean_absolute_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


In [54]:
data['higher'] = data['<CLOSE>'] - data['<OPEN>'] > 0
le = LabelEncoder()
data['higher'] = le.fit_transform(data['higher'])

In [ ]:
tickers = data['<TICKER>'].unique()

accuracy = {}

for ticker in tickers:

    ticker_data = data[data['<TICKER>'] == ticker].copy()
    ticker_data['prev_open'] = ticker_data['<OPEN>'].shift(14)
    ticker_data['prev_high'] = ticker_data['<HIGH>'].shift(14)
    ticker_data['prev_close'] = ticker_data['<CLOSE>'].shift(14)
    ticker_data['prev_close_7'] = ticker_data['<CLOSE>'].shift(7)
    ticker_data['prev_low'] = ticker_data['<LOW>'].shift(14)
    ticker_data['prev_vol'] = ticker_data['<VOL>'].shift(14)
    ticker_data = ticker_data.drop(ticker_data.index[:14])
    ticker_data = ticker_data.reset_index(drop=True)

    X = ticker_data[['prev_open', 'prev_high','prev_close_7', 'prev_close', 'prev_low', 'prev_vol', '<OPEN>']]
    y = ticker_data['higher']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False, stratify = None)

    model = LogisticRegression()
    param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 0.9],
                'class_weight': [None, 'balanced'],
                'penalty': ['l1', 'l2']
                }


    model = LogisticRegression()


    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=50, cv=3)
    random_search.fit(X, y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy[ticker] = {"Best Parameters": random_search.best_params_,
                        "Best Score": random_search.best_score_}
    


In [ ]:
accuracy

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
from sklearn.metrics import precision_score, make_scorer, recall_score

for ticker in tickers:

    ticker_data = data[data['<TICKER>'] == ticker].copy()
    ticker_data['prev_open'] = ticker_data['<OPEN>'].shift(14)
    ticker_data['prev_high'] = ticker_data['<HIGH>'].shift(14)
    ticker_data['prev_close'] = ticker_data['<CLOSE>'].shift(14)
    ticker_data['prev_close_7'] = ticker_data['<CLOSE>'].shift(7)
    ticker_data['prev_low'] = ticker_data['<LOW>'].shift(14)
    ticker_data['prev_vol'] = ticker_data['<VOL>'].shift(14)
    ticker_data = ticker_data.drop(ticker_data.index[:14])
    ticker_data = ticker_data.reset_index(drop=True)

    X = ticker_data[['prev_open', 'prev_high','prev_close_7', 'prev_close', 'prev_low', 'prev_vol', '<OPEN>']]
    y = ticker_data['higher']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False, stratify = None)

    param_grid = {'n_neighbors': randint(1, 10),
                'weights': ['uniform', 'distance'],
                'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                'leaf_size': randint(10, 50),
                'p': [1, 2],
                'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}


    model = KNeighborsClassifier()
    scorer = make_scorer(precision_score)


    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, scoring=scorer)
    random_search.fit(X, y)

    print("Best Parameters:", random_search.best_params_)
    print("Best Score:", random_search.best_score_)
    y_pred = random_search.predict(X)
    accuracy[ticker] = [precision_score(y, y_pred), recall_score(y, y_pred)]

In [58]:
accuracy

{'AFLT': [1.0, 1.0],
 'ALRS': [1.0, 1.0],
 'GAZP': [1.0, 1.0],
 'GMKN': [0.7973549488054608, 0.48545454545454547],
 'HYDR': [1.0, 1.0],
 'IRAO': [0.7210657785179018, 0.4613745338305807],
 'LKOH': [0.692667145938174, 0.4993521637729982],
 'MGNT': [0.740293134481872, 0.7404835390946503],
 'MOEX': [1.0, 1.0],
 'MTLR': [1.0, 1.0],
 'MTSS': [1.0, 1.0],
 'NLMK': [0.6908998474834773, 0.6887987835783071],
 'ROSN': [1.0, 1.0],
 'RTSI': [1.0, 1.0],
 'SBER': [0.7211908931698774, 0.5159107992984214],
 'SNGS': [1.0, 1.0],
 'VTBR': [0.8062943262411347, 0.4832624867162593],
 'YNDX': [0.6838589981447124, 0.4805736636245111]}